# Atheris API

#### `Setup(args, test_one_input, internal_libfuzzer=None)`

 - `args`: A list of strings: the process arguments to pass to the fuzzer, typically `sys.argv`. 
  
    This argument list may be modified in-place, to remove arguments consumed by the fuzzer.

    See [the LibFuzzer docs](https://llvm.org/docs/LibFuzzer.html#options) for a list of such options.
    
    通常通过命令行参数传入，定义了fuzzing engine的行为。

 - `test_one_input`: your fuzzer's entry point. 
    
    Must take a single `bytes` argument. This will be repeatedly invoked with a single bytes container.

 - `internal_libfuzzer`: Indicates whether libfuzzer will be provided by atheris or by an external library (see [native_extension_fuzzing.md](./native_extension_fuzzing.md)). 
  
    If unspecified, Atheris will determine this
    automatically. If fuzzing pure Python, leave this as `True`.

In [ ]:
import atheris
import zlib
import sys


@atheris.instrument_func
def TestOneInput(data):
  try:
    decompressed = zlib.decompress(data)
  except zlib.error:
    return

  if len(decompressed) < 2:
    return

  try:
    if decompressed.decode() == 'FU':
      raise RuntimeError('Boom')
  except UnicodeDecodeError:
    pass
  
def main():
  atheris.Setup(sys.argv, TestOneInput)
  atheris.Fuzz()

if __name__ == '__main__':
    main()